# ChatQwen

This will help you get started with Qwen [chat models](../../concepts/chat_models.mdx). For detailed documentation of all ChatQwen features and configurations head to the [API reference](https://pypi.org/project/langchain-qwq/).

## Overview
### Integration details


| Class | Package | Local | Serializable | Package downloads | Package latest |
| :--- | :--- | :---: |  :---: | :---: | :---: |
| [ChatQwen](https://pypi.org/project/langchain-qwq/) | [langchain-qwq](https://pypi.org/project/langchain-qwq/) | ❌ | beta | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-qwq?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-qwq?style=flat-square&label=%20) |

### Model features
| [Tool calling](../../how_to/tool_calling.ipynb) | [Structured output](../../how_to/structured_output.ipynb) | JSON mode | [Image input](../../how_to/multimodal_inputs.ipynb) | Audio input | Video input | [Token-level streaming](../../how_to/chat_streaming.ipynb) | Native async | [Token usage](../../how_to/chat_token_usage_tracking.ipynb) | [Logprobs](../../how_to/logprobs.ipynb) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ |✅  | ❌ | ✅ | ✅ | ✅ | ✅ | ❌ | 

## Setup

To access Qwen models you'll need to create an Alibaba Cloud account, get an API key, and install the `langchain-qwq` integration package.

### Credentials

Head to [Alibaba's API Key page](https://account.alibabacloud.com/login/login.htm?oauth_callback=https%3A%2F%2Fbailian.console.alibabacloud.com%2F%3FapiKey%3D1&lang=en#/api-key) to sign up to Alibaba Cloud and generate an API key. Once you've done this set the `DASHSCOPE_API_KEY` environment variable:

In [1]:
import getpass
import os

if not os.getenv("DASHSCOPE_API_KEY"):
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter your Dashscope API key: ")

### Installation

The LangChain Qwen integration lives in the `langchain-qwq` package:

In [ ]:
%pip install -qU langchain-qwq

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [1]:
from langchain_qwq import ChatQwen


llm = ChatQwen(model="qwen-flash")
response = llm.invoke("Hello")

response

AIMessage(content='Hello! How can I help you today? 😊', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-flash'}, id='run--f68b2865-b969-47a0-ad1f-b88bb3b5906b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20, 'input_token_details': {}, 'output_token_details': {}})

## Invocation

In [2]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French."
        "Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-flash'}, id='run--40f2e75b-7d28-4a71-8f5f-561509ac2010-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40, 'input_token_details': {}, 'output_token_details': {}})

## Chaining

We can [chain](../../how_to/sequence.ipynb) our model with a prompt template like so:

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates"
            "{input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe Programmierung.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-flash'}, id='run--ed052510-82f4-4dc5-a7f1-761a792c0dd4-0', usage_metadata={'input_tokens': 28, 'output_tokens': 5, 'total_tokens': 33, 'input_token_details': {}, 'output_token_details': {}})

## Tool Calling
ChatQwen supports tool calling API that lets you describe tools and their arguments, and have the model return a JSON object with a tool to invoke and the inputs to that tool.

### Use with `bind_tools`

In [6]:
from langchain_core.tools import tool
from langchain_qwq import ChatQwen


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatQwen(model="qwen-flash")

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("What's 5 times forty two")

print(msg)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_f0c2cc49307f480db78a45', 'function': {'arguments': '{"first_int": 5, "second_int": 42}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen-flash'} id='run--27c5aafb-9710-42f5-ab78-5a2ad1d9050e-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_f0c2cc49307f480db78a45', 'type': 'tool_call'}] usage_metadata={'input_tokens': 166, 'output_tokens': 27, 'total_tokens': 193, 'input_token_details': {}, 'output_token_details': {}}


### vision Support

#### Image

In [ ]:
from langchain_core.messages import HumanMessage

model = ChatQwen(model="qwen-vl-max-latest")

messages = [
    HumanMessage(content=[
        {
            "type": "image_url",
            "image_url": {
                "url": "https://example.com/image.jpg"
            },
        },
        {"type": "text", "text": "What do you see in this image?"},
    ])
]

response = model.invoke(messages)
print(response)

content='In this image, I see a heartwarming scene on a sandy beach during what appears to be either sunrise or sunset, given the warm, soft golden light. A woman is sitting on the sand, smiling warmly as she gently holds the paw of a large, light-colored dog—likely a Labrador Retriever. The dog is sitting upright, facing her, and seems to be engaging in a friendly handshake or high-five gesture with her. \n\nThe woman is wearing a plaid shirt and dark pants, and her long hair flows loosely. She looks happy and relaxed, enjoying the moment with her pet. The dog is wearing a colorful harness with patterns, and a leash lies loosely on the sand nearby. \n\nIn the background, gentle waves roll onto the shore, and the horizon blends softly into the sky, creating a serene and peaceful atmosphere. The overall mood of the image is joyful, affectionate, and tranquil, capturing a special bond between the woman and her dog in a beautiful natural setting.' additional_kwargs={} response_metadata={'

#### Video

In [ ]:
from langchain_core.messages import HumanMessage

model = ChatQwen(model="qwen-vl-max-latest")

messages = [
    HumanMessage(content=[
        {
            "type": "video_url",
            "video_url": {
                "url": "https://example.com/video.mp4"
            },
        },
        {"type": "text", "text": "Can you tell me about this video?"},
    ])
]

response = model.invoke(messages)
print(response)

content='This video features a young woman with short, neatly styled brown hair and bangs. She is wearing a soft pink knitted cardigan over a white top, accessorized with a delicate necklace. The background is softly blurred, suggesting an outdoor urban setting with modern buildings, possibly during daytime with natural lighting.\n\nThroughout the video, the woman maintains a warm and friendly demeanor, smiling gently and occasionally laughing. Her expressions are animated and cheerful, conveying a sense of happiness and approachability. The lighting highlights her features beautifully, giving the scene a pleasant and inviting atmosphere.\n\nThe watermark in the top-right corner indicates that this video was created using AI synthesis technology ("通义·AI合成"), suggesting that the content may be generated or enhanced by artificial intelligence. This could mean the visuals are highly polished and stylized, typical of AI-generated media.\n\nOverall, the video appears to be a short, positive

## API reference

For detailed documentation of all ChatQwen features and configurations head to the [API reference](https://pypi.org/project/langchain-qwq/)